In [1]:
import sys
import os
from pathlib import Path

from IPython.display import display, Markdown
from openai import OpenAI
from dotenv import dotenv_values

# Add the parent directory to sys.path so Python can find the Utils module
sys.path.append(os.path.abspath(".."))

from Utils import (
    get_guidance_and_feedback,
    get_questions_and_answers,
    create_content,
)

# Get required environment variables
config = dotenv_values("../.env")
open_api_key = config.get("OPENAI_API_KEY", None)
tutor_agent_id = config.get("TUTOR_AGENT_ID", None)

# Check if the environment variables are set
if not (tutor_agent_id and open_api_key):
    print("Need TUTOR_AGENT_ID and OPEN_API_KEY in the .env file")
    sys.exit(1)

# Create the OpenAI client
client = OpenAI(api_key=config["OPENAI_API_KEY"])

Generate the content for our prompt message to our bot.  Our prompt must look like the input examples we provided to our assistant in its instructions.  For this toy example we are using JSON files of question data and student answers as a store.

In [2]:
q_path = Path('../Data/question_store.json')
a_path = Path('../Data/student_answers.json')
questions, answers = get_questions_and_answers(q_path, a_path)

Having loaded the question and answer data we can now create input content.

In [6]:
question_id = "Q3"
student_id = "student_8"

question = questions[question_id]
student_answer = answers[question_id][student_id]

content = create_content(question, student_answer)
print(content)

Input:
Exam Question: "Given, $E_v = h\\nu(v+1/2)$ explain the terms in this equation. Explain how this equation is developed, invoking our 5 fundamental rules of quantum mechanics. (You are not expected to do an explicit mathematical derivation)."
Student Answer: "$h$ is planck's constant, $E_v$ is energy, v is quantum number and $\\nu$ is the frequency. The equation is based on a classical physics equation and variables that can be observed or replaced with quantum numbers. The Schrodinger equation is then built and solved, followed by boundary conditions being added."
Concepts Tested: 
	- Recognition of eigenvalue for simple harmonic oscillator.
	- Knowledge of five key rules in the application of quantum mechanics to physical systems. 1) Classical Description: Define the system's energy using classical physics. 2) Quantum Operators: Replace each physical observable (e.g., momentum, position) with its corresponding quantum mechanical operator. 3) Hamiltonian Development: Construct t

Now we start a new conversation thread and send our content as a messages to our assistant.  This can take some time to run.

In [7]:
thread, messages, run = get_guidance_and_feedback(
    client, content, tutor_agent_id, temperature=0.1, thread_id=None, show_run=True)

**Bot Message**:


Your answer provides a basic understanding of the terms in the equation $E_v = h\nu(v+1/2)$, but it lacks depth in explaining how these terms relate to the quantum mechanical description of a simple harmonic oscillator and the application of the five fundamental rules of quantum mechanics.

1. **Correct Elements in Your Answer:**
   - You correctly identified $h$ as Planck's constant, $E_v$ as energy, $v$ as the quantum number, and $\nu$ as the frequency.
   - You mentioned the Schrödinger equation and boundary conditions, which are indeed part of the process.

2. **Areas Needing Improvement:**
   - **Connection to Simple Harmonic Oscillator:** The equation $E_v = h\nu(v+1/2)$ specifically describes the energy levels of a quantum mechanical harmonic oscillator, not a general energy equation derived from classical physics. This distinction is crucial as it highlights the quantization of energy levels.
   - **Detailed Application of Quantum Mechanics Rules:**
     - **Classical Description:** You should explicitly state that the classical analogy for the harmonic oscillator helps in forming the quantum description.
     - **Quantum Operators:** Mention how physical observables like position and momentum are replaced by their corresponding quantum operators.
     - **Hamiltonian Development:** Discuss how these operators are used to construct the Hamiltonian for the harmonic oscillator.
     - **Schrödinger Equation:** Explain that solving this equation with the Hamiltonian gives the wave functions and energy eigenvalues.
     - **Boundary Conditions:** Elaborate on how these ensure the physical acceptability of the wave functions, leading to quantized energy levels.
   - **Depth and Precision:** Your answer should more precisely connect these rules to how they lead to the derivation of the given energy equation for the harmonic oscillator.

3. **Textbook Sections for Review:**
   - Initially, I would recommend reviewing Chapter 5, Section 5.6 on the Harmonic Oscillator. However, if you find that section not as helpful, please let me know, and I will provide a more accurate reference.

By revisiting these sections and focusing on the specific application of quantum mechanics to the harmonic oscillator, you can enhance your understanding and ability to explain these concepts more thoroughly.



**Run object attributes**:
- status: completed
- temperature: 0.1
- tools: [FileSearchTool(type='file_search')]
- usage: Usage(completion_tokens=466, prompt_tokens=3817, total_tokens=4283)


Now we have started the thread, we can continue if we want the chat with the bot.

In [6]:
follow_up = "I can't find any information on orbiting electrons and current in section 3.1. Please can you look again in the textbook and tell me where to look?"
thread, messages, run = get_guidance_and_feedback(
    client, follow_up, tutor_agent_id, temperature=0.1, thread_id=thread, show_run=True)

**Bot Message**:


For a detailed explanation on the motion of an electric charge around a loop and how it generates a magnetic moment, you can refer to Chapter 7, specifically section 7.4 in the textbook "Quantum Chemistry" by Donald A. Macquarrie. This section explains the relationship between current, magnetic moment, and the motion of charges in circular paths, which is directly relevant to your question about the current generated by an electron in orbit.

Here's the specific citation from the textbook:
- Chapter 7.4, "The Energy Levels of a Hydrogen Atom Are Split by a Magnetic Field" [0].

This section will provide you with the necessary background and formulas to understand and calculate the current generated by an orbiting electron, including how to handle the charge and velocity of the electron in the context of magnetic fields and circular motion.
[0] None from QUANTUM_CHEMISTRY_SECOND_EDITION.pdf


**Run object attributes**:
- status: completed
- temperature: 0.1
- tools: [FileSearchTool(type='file_search')]
- usage: Usage(completion_tokens=206, prompt_tokens=24057, total_tokens=24263)


In [ ]:
follow_up = "Which section of the textbook explains the second postulate of quantum mechanics?"
thread, messages, run = get_guidance_and_feedback(
    client, follow_up, tutor_agent_id, temperature=0.1, thread_id=thread, show_run=True)

**Bot Message**:


The second postulate of quantum mechanics, which involves replacing classical observables with quantum mechanical operators, is typically discussed in the context of the formulation of quantum mechanics principles. In the textbook "Quantum Chemistry" by Donald A. Macquarrie, you can find detailed explanations of this postulate in:

- **Chapter 3**, specifically in the sections that discuss quantum operators and their application in quantum mechanics.

This section will help you understand how physical observables are represented as operators in the quantum mechanical framework, which is crucial for formulating the Hamiltonian and solving the Schrödinger equation.



**Run object attributes**:
- status: completed
- temperature: 0.1
- tools: [FileSearchTool(type='file_search')]
- usage: Usage(completion_tokens=123, prompt_tokens=4269, total_tokens=4392)


In [18]:
import json

json_data = '''
{
    "example": "This is an example $\\\\times$ with LaTeX and escaped characters."
}
'''

# Correctly parse JSON from a string
data = json.loads(json_data)

# Print the result
print(data['example'])


This is an example $\times$ with LaTeX and escaped characters.
